In [1]:
from estnltk import Text
import csv
from datetime import datetime
import psycopg2

In [2]:
from taggers.drug_tagger import DrugFieldPartTagger, DrugTagger
from cda_data_cleaning.common.db_operations import create_connection
from cda_data_cleaning.common import read_config

In [3]:
part_tagger = DrugFieldPartTagger()

In [4]:
drug_tagger = DrugTagger()

In [27]:
config_file = 'egcut_epi.ini'
config = read_config(config_file)

conn = create_connection(config)
cur = conn.cursor()

In [28]:
cur.execute("""SELECT * from original.drug order by epi_id limit 1000""")

In [29]:
drug_rows = cur.fetchall()

In [30]:
new_rows = []
for row in drug_rows:
    if row[3]:
        t = Text(row[3]).tag_layer(['words'])
        part_tagger.tag(t)
        drug_tagger.tag(t)

        for j in t.parsed_drug:
            date = ''
            atc_code = ''
            drug_name = ''
            active_ingredient = ''
            if j.date:
                date = j.date
            if j.atc_code:
                atc_code = j.atc_code
            if j.drug_name:
                drug_name = j.drug_name
            if j.active_ingredient:    
                active_ingredient = j.active_ingredient
            new_rows.append([row[0], row[1], row[2], date, atc_code, drug_name, active_ingredient])    

In [31]:
len(new_rows)

1500

In [32]:
len(drug_rows)

1000

In [33]:
new_rows[:100]

[[1, '1000222', 's', '', 'B01AC04', '', ''],
 [1, '1000222', 's', '', 'C09AA05', '', ''],
 [1, '1000222', 's', '', 'C10AA07', '', ''],
 [2, '1000285', 's', '', 'C01DA14', '', ''],
 [2, '1000285', 's', '', 'C10AA07', '', ''],
 [2, '1000285', 's', '', 'C07AB02', '', ''],
 [3, '1000298', 's', '', 'C01DA14', '', ''],
 [3, '1000298', 's', '', 'B01AC04', '', ''],
 [3, '1000298', 's', '', 'C07AB02', '', ''],
 [3, '1000298', 's', '', 'C09BA09', '', ''],
 [3, '1000298', 's', '', 'N05BA12', '', ''],
 [4, '1000402', 's', '', 'C01DA14', '', ''],
 [4, '1000402', 's', '', 'C09AA05', '', ''],
 [4, '1000402', 's', '', 'C03CA01', '', ''],
 [4, '1000402', 's', '', 'B01AC04', '', ''],
 [5, '1000420', 's', '', 'C01BD01', '', ''],
 [7, '1000482', 's', '', 'C10AA01', '', ''],
 [7, '1000482', 's', '', 'B01AC04', '', ''],
 [7, '1000482', 's', '', 'C09CA01', '', ''],
 [7, '1000482', 's', '', 'C07AB02', '', ''],
 [83469, '10005419', 'a', ' 13.10.2010,', 'H03BA02', 'TIOTIL', ''],
 [83469, '10005419', 'a', ' 13.1

In [34]:
cur.execute("""set role egcut_epi_work_create;""")

In [35]:
cur.execute("""drop table work.drug_parsed;""")

In [36]:
cur.execute("""create table work.drug_parsed (id bigint, epi_id text, 
epi_type text, date text, ATC text, drug_name text, active_ingredient text);""")

In [37]:
for row in new_rows:
    cur.execute("insert into work.drug_parsed VALUES (%s, %s, %s, %s, %s, %s, %s)",  (row[0], row[1], row[2], row[3], row[4], row[5], row[6]))

In [26]:
conn.commit()
cur.close()
conn.close()

In [24]:
for r in drug_rows:
    if r[1] == '10458531':
        print(r)

(320, '10458531', 's', 'Väljastamise kuupäev: 20.11.2012, Retsepti number:  1015511113, Ravimi nimetus: DIAPREL MR 60MG 60TK, Originaalide arv: 1, ATC kood: A10BB09, Toimeaine: Gliclazidum, Ravimvorm: toimeainet modifitseeritult vabastav tablett, Soodustus: 90%, Annustamine:  , Annustamine täpsemalt: ½ tabletti enne hommikusööki.  Väljastamise kuupäev: 20.11.2012, Retsepti number:  2015511113, Ravimi nimetus: DIAPREL MR 60MG 60TK, Originaalide arv: 1, ATC kood: A10BB09, Toimeaine: Gliclazidum, Ravimvorm: toimeainet modifitseeritult vabastav tablett, Soodustus: 90%, Annustamine:  , Annustamine täpsemalt: ½ tabletti enne hommikusööki.  Väljastamise kuupäev: 20.11.2012, Retsepti number:  3015511113, Ravimi nimetus: DIAPREL MR 60MG 60TK, Originaalide arv: 1, ATC kood: A10BB09, Toimeaine: Gliclazidum, Ravimvorm: toimeainet modifitseeritult vabastav tablett, Soodustus: 90%, Annustamine:  , Annustamine täpsemalt: ½ tabletti enne hommikusööki.  Väljastamise kuupäev: 22.11.2012, Retsepti number

Text(text='Väljastamise kuupäev: 20.11.2012, Retsepti number:  1015511113, Ravimi nimetus: DIAPREL MR 60MG 60TK, Originaalide arv: 1, ATC kood: A10BB09')

In [32]:
from grammarextractor.taggers import DateTagger

In [33]:
d = DateTagger()

In [34]:
d.tag(t)

Text(text='Väljastamise kuupäev: 20.11.2012, Retsepti number:  1015511113, Ravimi nimetus: DIAPREL MR 60MG 60TK, Originaalide arv: 1, ATC kood: A10BB09')

In [35]:
t.dates_numbers

Layer(name='dates_numbers', attributes=('grammar_symbol', 'regex_type', 'value', '_priority_'), spans=SL[Span('20.11.2012', [{'grammar_symbol': 'DATE', 'regex_type': 'date4', 'value': 'date', '_priority_': -1}]),
Span('551111', [{'grammar_symbol': 'DATE', 'regex_type': 'date5', 'value': 'date', '_priority_': 4}]),
Span('511113', [{'grammar_symbol': 'DATE', 'regex_type': 'date5', 'value': 'date', '_priority_': 4}])])

In [38]:
from event_extraction.taggers import EventDateTagger

ModuleNotFoundError: No module named 'event_extraction'

In [44]:
from cda_data_cleaning.fact_extraction.measurement_extraction.taggers.robust_date_number_tagger.robust_date_number_tagger import RobustDateNumberTagger

In [45]:
r = RobustDateNumberTagger()

In [49]:
r.tag(t)

Text(text='Väljastamise kuupäev: 20.11.2012, Retsepti number:  1015511113, Ravimi nimetus: DIAPREL MR 60MG 60TK, Originaalide arv: 1, ATC kood: A10BB09')

In [50]:
t.dates_numbers

Layer(name='dates_numbers', attributes=('grammar_symbol', 'regex_type', 'value', '_priority_'), spans=SL[Span('20.11.2012', [{'grammar_symbol': 'DATE', 'regex_type': 'date4', 'value': 'date', '_priority_': -1}]),
Span('1015511113', [{'grammar_symbol': 'NUMBER', 'regex_type': 'anynumber', 'value': '1015511113', '_priority_': 1}]),
Span('60', [{'grammar_symbol': 'NUMBER', 'regex_type': 'anynumber', 'value': '60', '_priority_': 1}]),
Span('60', [{'grammar_symbol': 'NUMBER', 'regex_type': 'anynumber', 'value': '60', '_priority_': 1}]),
Span('1', [{'grammar_symbol': 'NUMBER', 'regex_type': 'anynumber', 'value': '1', '_priority_': 1}]),
Span('10', [{'grammar_symbol': 'NUMBER', 'regex_type': 'anynumber', 'value': '10', '_priority_': 1}]),
Span('09', [{'grammar_symbol': 'NUMBER', 'regex_type': 'anynumber', 'value': '09', '_priority_': 1}])])

In [52]:
from cda_data_cleaning.fact_extraction.event_extraction.taggers.event_date_tagger import EventDateTagger

In [53]:
e = EventDateTagger()

In [56]:
e.tag(t)

Text(text='Väljastamise kuupäev: 20.11.2012, Retsepti number:  1015511113, Ravimi nimetus: DIAPREL MR 60MG 60TK, Originaalide arv: 1, ATC kood: A10BB09')

In [58]:
t.event_dates

Layer(name='event_dates', attributes=('grammar_symbol', 'regex_type', 'value', '_priority_'), spans=SL[])